In [1]:
import re
import pandas as pd
import numpy as np
import itertools
import copy

# Input(s)

In [729]:
#My input, the rest are for testing purposes
inp = """G=(Vn, Vt, P, S)
        Vn={S, A, B, C, D}
        Vt={a, b}
        P={
        S->aB
        S->A
        A->bAa
        A->aS
        A->a
        B->AbB
        B->BS
        B->a
        B->ε
        C->BA
        C->AAA
        D->a
        }
         """

In [676]:
inp = """G=(VN, VT, P, S)
        Vn={S, A, B, C, D}
        Vt={a, b}
        P={
        S->bA
        S->BC
        A->a
        A->aS
        A->bAaAb
        B->A
        B->bS
        B->aAa
        C->ε
        C->AB
        D->AB
        }
        """

# Grammar Class

In [730]:
class Grammar():
    def __init__(self):
        self.S = ""
        self.Vn = []
        self.Vt = []
        self.P = {}
        
    def read_input(self, inp):
        def iterator(string):
            li = []
            symbol = ""
            for item in string:
                if item != "}" and item != ")":
                    li.append(item)
                else:
                    break
            return li
        
        def construct_P(string):
            p = iterator(string)
            lhs = p[0::2]
            rhs = p[1::2]
            
            for i in range(len(lhs)):
                if lhs[i] not in self.P.keys():
                    self.P[lhs[i]] = [rhs[i]]
                else:
                    self.P[lhs[i]].append(rhs[i])
            
        grammar = re.findall("\w+|\(|\)|{|}", inp)
        
        for i in range(len(grammar)):
            if grammar[i] == "G":
                self.S = grammar[i + 5]
            elif grammar[i] == "Vn":
                self.Vn = iterator(grammar[i+2:])
            elif grammar[i] == "Vt":
                self.Vt = iterator(grammar[i+2:])
            elif grammar[i] == "P":
                construct_P(grammar[i+2:])

# Stage0: Ensure no S on RHS

In [731]:
class Stage0():
    def __init__(self):
        pass
        
    def apply(self, grammar):
        grammar.P["S_0"] = [grammar.S, "ε"]
        grammar.Vn.append("S_0")
        grammar.S = "S_0"

# Stage1: Remove Empty Strings

In [732]:
class Stage1():
    def __init__(self, grammar):
        self.S = grammar.S
        self.Vn = grammar.Vn
        self.Vt = grammar.Vt
        self.P = grammar.P
        
    def get_nullables(self):
        derives_in = ["ε"]
        nullables = []
        while True:
            flag = 0
            for node in self.Vn:
                for d in derives_in:
                    for element in self.P[node]:
                        for character in element:
                            if character == d and node not in nullables:
                                flag += 1
                                nullables.append(node)
                                derives_in.append(node)
            if flag == 0:
                break
        return nullables
    
    def get_all_poss_combinations(self, string):
        nullables = self.get_nullables()
        def foo(str_len):
             yield from itertools.product(*([["0", "1"]] * str_len))
                
        l = 0
        for char in string:
            if char in nullables:
                l += 1
        binary_combs = foo(l)
        
        comb_list = []
        for tup in binary_combs:
            curr_str = ""
            ind = 0
            for char in string:
                if char in nullables:
                    if tup[ind] == "1":
                        curr_str += char
                        ind += 1
                    else:
                        ind += 1
                else:
                    curr_str += char
            if curr_str != "":
                comb_list.append(curr_str)
            
        return comb_list
    
    def apply(self):
        P_copy = copy.deepcopy(self.P)
        nullables = self.get_nullables()
        
        for key in P_copy.keys():
            for string in P_copy[key]:
#                 print("here lol " + string)
                for char in string:
                    if char in nullables:
                        self.P[key].remove(string)
                        self.P[key] += self.get_all_poss_combinations(string)
                        break
        
        for key in self.P.keys():
            self.P[key] = list(set(self.P[key]))
            if "ε" in self.P[key] and key != self.S:
                self.P[key].remove("ε")

# Stage2: Remove renamings

In [733]:
class Stage2():
    def __init__(self):
        pass
    
    def apply(self, grammar):
        self.grammar = grammar
        P_copy = copy.deepcopy(grammar.P)
        
        try:
            for key in P_copy.keys():
                for string in P_copy[key]:
                    if string in grammar.Vn:
                        grammar.P[key].remove(string)
                        grammar.P[key] += grammar.P[string]
                        self.apply(self.grammar)
        except:
            pass
        
        for key in grammar.P.keys():
            grammar.P[key] = list(set(grammar.P[key]))

# Stage3: Remove Unproductive

In [734]:
class Stage3():
    def __init__(self):
        pass
    
    def get_all_unproductive(self, grammar):
        productive = []
#         print(grammar.Vt)
        for key in grammar.P.keys():
            for string in grammar.P[key]:
                cnt = 0
                for character in string:
                    if character in grammar.Vt:
                        cnt += 1
                if cnt == len(string):
                    productive.append(key)
                    break
                    
        while True:
            productive_len = len(productive)
            for key in grammar.P.keys():
                if key not in productive:
                    for string in grammar.P[key]:
                        for char in string:
                            flag = 0
                            if char in grammar.Vn and char not in productive:
                                break
                            flag = 1
                        if flag == 1:
                            productive.append(key)
                            break
                            
#             print("While loop ran at least once")
#           while break condition
            curr_productive_len = len(productive)
            if curr_productive_len == productive_len:
                break
        
        unproductive = []
        
        for element in grammar.Vn:
            if element not in productive:
                unproductive.append(element)
                            
        return unproductive
    
    def apply(self, grammar):
        unproductive = self.get_all_unproductive(grammar)
        
        for element in unproductive:
            grammar.P.pop(element)
            if element in grammar.Vn:
                grammar.Vn.remove(element)
            else:
                grammar.Vt.remove(element)
            
        for key in grammar.P.keys():
            for string in grammar.P[key]:
                for char in string:
                    if char in unproductive:
                        grammar.P[key].remove(string)

# Stage4: Remove Innaccssible

In [735]:
class Stage4():
    def __init__(self):
        pass
    
    def get_all_innaccessible(self, grammar):
        accessible = [grammar.S]
        start = [grammar.S]
        
        while True:
            for element in start:
                start = []
                if element in grammar.Vn:
                    for string in grammar.P[element]:
                        for character in string:
                            if character not in accessible:
                                accessible.append(character)
                                start.append(character)
                            
            if len(start) == 0:
                break
        
        inaccessible = []
        for element in (grammar.Vn + grammar.Vt):
            if element not in accessible:
                inaccessible.append(element)
                
        return inaccessible
    
    def apply(self, grammar):
        inaccessible = self.get_all_innaccessible(grammar)
        
        for element in inaccessible:
            grammar.P.pop(element)
            if element in grammar.Vn:
                grammar.Vn.remove(element)
            else:
                grammar.Vt.remove(element)
            
        for key in grammar.P.keys():
            for string in grammar.P[key]:
                for char in string:
                    if char in inaccessible:
                        grammar.P[key].remove(string)

# Stage5: Normalize

In [736]:
class Step5():
    def __init__(self):
        pass
    
    def replace_terminals(self, grammar):
        X_list = []
        X_goes_to = []
        
        for key in grammar.P.keys():
            for i in range(len(grammar.P[key])):
                if len(grammar.P[key][i]) > 1:
                    for character in grammar.P[key][i]:
                        if character in grammar.Vt and character not in X_goes_to:
                            X_list.append("X_" + str(len(X_list)))
                            X_goes_to.append(character)
                            grammar.P[key][i] = (grammar.P[key][i]).replace(character, X_list[-1])
                        elif character in grammar.Vt and character in X_goes_to:
                            index = X_goes_to.index(character)
                            grammar.P[key][i] = (grammar.P[key][i]).replace(character, X_list[index])
                            
        for i in range(len(X_list)):
            grammar.P[X_list[i]] = [X_goes_to[i]]
            grammar.Vn.append(X_list[i])
                            
    def replace_nonterminals(self, grammar):
        Y_list = []
        Y_goes_to = []
        
        while True:
            Y_list_len = len(Y_list)
            for key in grammar.P.keys():
                for i in range(len(grammar.P[key])):
                    symbol_list = re.findall("X_[0-9]+|Y_[0-9]+|[A-Z]", grammar.P[key][i])
                    if len(symbol_list) > 2:
                        for j in range(0, len(symbol_list)-1, 2):
                            if (symbol_list[j], symbol_list[j+1]) not in Y_goes_to:
                                Y_list.append("Y_" + str(len(Y_list)))
                                Y_goes_to.append((symbol_list[j], symbol_list[j+1]))
                                grammar.P[key][i] = (grammar.P[key][i]).replace(symbol_list[j]+symbol_list[j+1], Y_list[-1])
                            else:
                                index = Y_goes_to.index((symbol_list[j], symbol_list[j+1]))
                                grammar.P[key][i] = (grammar.P[key][i]).replace(symbol_list[j]+symbol_list[j+1], Y_list[index])
                                
            Y_list_len_curr = len(Y_list)
            
            if Y_list_len == Y_list_len_curr:
                break
                
        for i in range(len(Y_list)):
            grammar.P[Y_list[i]] = ["".join([Y_goes_to[i][0], Y_goes_to[i][1]])]
            grammar.Vn.append(Y_list[i])
                
    def apply(self, grammar):
        self.replace_terminals(grammar)
        self.replace_nonterminals(grammar)

In [737]:
g = Grammar()

In [738]:
g.read_input(inp)
g.P

{'S': ['aB', 'A'],
 'A': ['bAa', 'aS', 'a'],
 'B': ['AbB', 'BS', 'a', 'ε'],
 'C': ['BA', 'AAA'],
 'D': ['a']}

In [739]:
st0 = Stage0()
st0.apply(g)

In [740]:
st1 = Stage1(g)

In [741]:
st1.apply()

In [742]:
st1.P

{'S': ['aB', 'A', 'a'],
 'A': ['ba', 'aS', 'bAa', 'a'],
 'B': ['AbB', 'B', 'Ab', 'bB', 'a', 'b', 'BS', 'S'],
 'C': ['B', 'A', 'BA', 'AAA', 'AA'],
 'D': ['a'],
 'S_0': ['ε', 'S']}

In [743]:
g.P

{'S': ['aB', 'A', 'a'],
 'A': ['ba', 'aS', 'bAa', 'a'],
 'B': ['AbB', 'B', 'Ab', 'bB', 'a', 'b', 'BS', 'S'],
 'C': ['B', 'A', 'BA', 'AAA', 'AA'],
 'D': ['a'],
 'S_0': ['ε', 'S']}

In [744]:
st2 = Stage2()
st2.apply(g)

In [745]:
g.P

{'S': ['aB', 'a', 'aS', 'ba', 'bAa'],
 'A': ['ba', 'bAa', 'aS', 'a'],
 'B': ['aB', 'AbB', 'Ab', 'a', 'b', 'bB', 'aS', 'BS', 'ba', 'bAa'],
 'C': ['aB',
  'AbB',
  'AAA',
  'Ab',
  'bB',
  'a',
  'b',
  'aS',
  'BA',
  'BS',
  'ba',
  'bAa',
  'AA'],
 'D': ['a'],
 'S_0': ['aB', 'ε', 'a', 'aS', 'ba', 'bAa']}

In [746]:
st3 = Stage3()
st3.get_all_unproductive(g)

[]

In [747]:
st3.apply(g)

In [748]:
g.P

{'S': ['aB', 'a', 'aS', 'ba', 'bAa'],
 'A': ['ba', 'bAa', 'aS', 'a'],
 'B': ['aB', 'AbB', 'Ab', 'a', 'b', 'bB', 'aS', 'BS', 'ba', 'bAa'],
 'C': ['aB',
  'AbB',
  'AAA',
  'Ab',
  'bB',
  'a',
  'b',
  'aS',
  'BA',
  'BS',
  'ba',
  'bAa',
  'AA'],
 'D': ['a'],
 'S_0': ['aB', 'ε', 'a', 'aS', 'ba', 'bAa']}

In [749]:
st4 = Stage4()
st4.get_all_innaccessible(g)

['C', 'D']

In [750]:
st4.apply(g)

In [751]:
g.P

{'S': ['aB', 'a', 'aS', 'ba', 'bAa'],
 'A': ['ba', 'bAa', 'aS', 'a'],
 'B': ['aB', 'AbB', 'Ab', 'a', 'b', 'bB', 'aS', 'BS', 'ba', 'bAa'],
 'S_0': ['aB', 'ε', 'a', 'aS', 'ba', 'bAa']}

In [752]:
g.Vn

['S', 'A', 'B', 'S_0']

In [753]:
st5 = Step5()
st5.replace_terminals(g)

In [754]:
g.P

{'S': ['X_0B', 'a', 'X_0S', 'X_1X_0', 'X_1AX_0'],
 'A': ['X_1X_0', 'X_1AX_0', 'X_0S', 'a'],
 'B': ['X_0B',
  'AX_1B',
  'AX_1',
  'a',
  'b',
  'X_1B',
  'X_0S',
  'BS',
  'X_1X_0',
  'X_1AX_0'],
 'S_0': ['X_0B', 'ε', 'a', 'X_0S', 'X_1X_0', 'X_1AX_0'],
 'X_0': ['a'],
 'X_1': ['b']}

In [755]:
g.Vn

['S', 'A', 'B', 'S_0', 'X_0', 'X_1']

In [756]:
st5.replace_nonterminals(g)

In [757]:
g.P

{'S': ['X_0B', 'a', 'X_0S', 'X_1X_0', 'Y_0X_0'],
 'A': ['X_1X_0', 'Y_0X_0', 'X_0S', 'a'],
 'B': ['X_0B',
  'Y_1B',
  'AX_1',
  'a',
  'b',
  'X_1B',
  'X_0S',
  'BS',
  'X_1X_0',
  'Y_0X_0'],
 'S_0': ['X_0B', 'ε', 'a', 'X_0S', 'X_1X_0', 'Y_0X_0'],
 'X_0': ['a'],
 'X_1': ['b'],
 'Y_0': ['X_1A'],
 'Y_1': ['AX_1']}

In [758]:
g.Vn

['S', 'A', 'B', 'S_0', 'X_0', 'X_1', 'Y_0', 'Y_1']